*   **Action:** Read Excel Sheet to Dataframe
*   **Sources:** `G:\My Drive\stocks\finviz_scrape.xlsm, Sheet:(S/E)yyyymmdd`
*   **Destinations:** `c:\Users\ping\Files_win10\python\py310\stocks\temp\df_finviz_(S/E)yyyymmdd.pkl`

In [6]:
import pandas as pd
import os
from IPython.display import display


# Configuration
FILE_PATH = r"G:\My Drive\stocks\finviz_scrape.xlsm"
DATE_SUFFIX = "2025-03-06"  # Now configurable
SHEET_PREFIXES = {'S': 'df_stocks', 'E': 'df_etfs'}

TEMP_DIR_PATH = os.path.join(os.getcwd(), '..\data')  # \data in parent directory

# Add date suffix to pickle filenames
PICKLE_STOCKS_FILE_NAME = f"df_finviz_stocks_{DATE_SUFFIX}.pkl"
PICKLE_ETFS_FILE_NAME = f"df_finviz_etfs_{DATE_SUFFIX}.pkl"

PICKLE_STOCKS_PATH = os.path.join(TEMP_DIR_PATH, PICKLE_STOCKS_FILE_NAME)
PICKLE_ETFS_PATH = os.path.join(TEMP_DIR_PATH, PICKLE_ETFS_FILE_NAME)


# Read and process data
dataframes = {}
for prefix in SHEET_PREFIXES:
    try:
        # Generate sheet name using prefix and date
        sheet_name = f"{prefix}{DATE_SUFFIX}"
        df = pd.read_excel(FILE_PATH, sheet_name=sheet_name)
        
        # Set index if Ticker column exists
        if 'Ticker' in df.columns:
            df = df.set_index('Ticker')
            print(f"Set 'Ticker' as index for {sheet_name}")
        else:
            print(f"Warning: 'Ticker' missing in {sheet_name}")
        
        # Store in both dictionary and named variable
        var_name = SHEET_PREFIXES[prefix]
        dataframes[var_name] = df
        globals()[var_name] = df
        print(f"Created {var_name} from {sheet_name}")
        
    except Exception as e:
        print(f"Error processing {sheet_name}: {str(e)}")


Set 'Ticker' as index for S2025-03-06
Created df_stocks from S2025-03-06
Set 'Ticker' as index for E2025-03-06
Created df_etfs from E2025-03-06


In [7]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\python310.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src']


In [8]:
import numpy as np
import pandas as pd

# Common helper functions
def process_metric_column(df, col_name, char_to_remove='B'):
    """Process columns containing metrics with B/commas"""
    df[col_name] = (
        df[col_name]
        .astype(str)
        .str.replace(char_to_remove, '', regex=False)
        .str.replace(',', '', regex=False)
        .apply(pd.to_numeric, errors='coerce')
    )
    return df

def clean_hyphens_and_convert(df, exclude_cols):
    """Replace hyphens with NaN and convert to numeric"""
    cols = df.columns.difference(exclude_cols)
    mask = df[cols].apply(lambda x: x.astype(str) == '-') 
    df[cols] = df[cols].mask(mask, np.nan).apply(pd.to_numeric, errors='coerce')
    return df

In [9]:
import numpy as np
import pandas as pd
import utils



# For df_stocks
df_stocks = process_metric_column(df_stocks, 'Market Cap')
df_stocks = clean_hyphens_and_convert(df_stocks, ['Industry'])
df_stocks['Industry'] = df_stocks['Industry'].convert_dtypes()

# For df_etfs, DO convert_volume to column 'Avg Volume' first,
# then except for 'AUM' and 'Tag', all other columns should be numbers
df_etfs['Avg Volume'] = df_etfs['Avg Volume'].apply(utils.convert_volume)

df_etfs = process_metric_column(df_etfs, 'AUM')
df_etfs = clean_hyphens_and_convert(df_etfs, ['Tags'])
df_etfs['Tags'] = df_etfs['Tags'].convert_dtypes()

# Display results (keep these separate as they show different outputs)
print("\nDataFrame for df_stocks\n")
display(df_stocks.info())
display(df_stocks)

print("\nDataFrame for df_etfs\n")
display(df_etfs.info())
display(df_etfs)


DataFrame for df_stocks

<class 'pandas.core.frame.DataFrame'>
Index: 1020 entries, AAPL to BXSL
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Industry       1020 non-null   string 
 1   Market Cap     1020 non-null   float64
 2   P/E            905 non-null    float64
 3   Fwd P/E        983 non-null    float64
 4   PEG            818 non-null    float64
 5   P/FCF          868 non-null    float64
 6   Dividend       762 non-null    float64
 7   Payout Ratio   933 non-null    float64
 8   EPS this Y     1011 non-null   float64
 9   EPS next Y     1009 non-null   float64
 10  EPS next 5Y    945 non-null    float64
 11  Sales past 5Y  1014 non-null   float64
 12  Sales Q/Q      1008 non-null   float64
 13  EPS Q/Q        1007 non-null   float64
 14  Inst Own       1017 non-null   float64
 15  ROA            1009 non-null   float64
 16  ROE            969 non-null    float64
 17  Curr R         908 non-null 

None

,Industry,Market Cap,P/E,Fwd P/E,PEG,P/FCF,Dividend,Payout Ratio,EPS this Y,EPS next Y,...,Quick R,Gross M,Oper M,Profit M,Beta,ATR,Volatility W,Volatility M,Recom,Target Price
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,Consumer Electronics,3535.14,37.41,28.72,3.80,35.96,0.0044,0.1611,0.0845,0.1192,...,0.88,0.4652,0.3176,0.2430,1.25,5.98,0.0320,0.0229,2.14,254.02
MSFT,Software - Infrastructure,2950.47,31.97,26.55,2.23,42.13,0.0083,0.2542,0.1146,0.1365,...,1.34,0.6941,0.4496,0.3543,0.99,9.37,0.0298,0.0197,1.29,508.26
NVDA,Semiconductors,2697.91,37.62,19.19,1.28,44.33,0.0004,0.0116,0.5194,0.2686,...,3.88,0.7499,0.6242,0.5585,1.94,7.21,0.0679,0.0493,1.28,175.11
AMZN,Internet Retail,2126.96,36.35,26.28,1.82,64.69,NaN,0.0000,0.1483,0.2027,...,0.87,0.4885,0.1087,0.0929,1.38,6.82,0.0404,0.0274,1.27,267.01
GOOGL,Internet Content & Information,2111.73,21.42,16.87,1.58,29.02,0.0029,0.0746,0.1090,0.1458,...,1.84,0.5826,0.3246,0.2862,1.01,4.99,0.0333,0.0248,1.55,219.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HALO,Biotechnology,7.41,17.52,8.94,0.67,15.82,NaN,0.0000,0.1962,0.3299,...,6.78,0.7730,0.5432,0.4374,1.40,1.57,0.0299,0.0261,2.00,66.11
BBWI,Specialty Retail,7.41,8.34,8.59,0.82,11.52,0.0249,0.2087,0.0944,0.1056,...,0.88,0.4397,0.1726,0.1243,1.66,1.74,0.0507,0.0414,1.63,45.25
MHK,"Furnishings, Fixtures & Appliances",7.40,14.53,10.17,1.43,10.88,NaN,0.0000,-0.0085,0.2085,...,1.12,0.2555,0.0728,0.0478,1.25,3.50,0.0292,0.0282,2.00,144.53



DataFrame for df_etfs

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, SPY to TDIV
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Dividend      400 non-null    float64
 1   Perf Quart    420 non-null    float64
 2   Perf Half     420 non-null    float64
 3   Beta          419 non-null    float64
 4   ATR           420 non-null    float64
 5   Volatility W  420 non-null    float64
 6   Volatility M  420 non-null    float64
 7   RSI           420 non-null    float64
 8   Avg Volume    420 non-null    float64
 9   Rel Volume    420 non-null    float64
 10  Expense       420 non-null    float64
 11  AUM           420 non-null    float64
 12  Flows% 1M     420 non-null    float64
 13  Flows% 3M     420 non-null    float64
 14  Flows% YTD    420 non-null    float64
 15  Return% 1Y    419 non-null    float64
 16  Return% 3Y    393 non-null    float64
 17  Return% 5Y    362 non-null    float64
 18  Tags    

None

,Dividend,Perf Quart,Perf Half,Beta,ATR,Volatility W,Volatility M,RSI,Avg Volume,Rel Volume,Expense,AUM,Flows% 1M,Flows% 3M,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
Ticker,,,,,,,,,,,,,,,,,,,
SPY,7.06,-0.0512,0.0374,1.01,9.00,0.0225,0.0131,34.03,51.73000,1.53,0.0009,619.80,0.0255,0.0209,-0.0021,0.1476,0.1162,0.1617,"U.S., equity, SP500"
VOO,6.70,-0.0511,0.0377,1.01,8.29,0.0225,0.0130,34.17,6.26000,1.15,0.0003,610.93,0.0153,0.0716,0.0544,0.1491,0.1172,0.1629,"U.S., equity, SP500, large-cap"
IVV,7.65,-0.0514,0.0365,1.01,9.04,0.0224,0.0130,34.10,6.64000,0.75,0.0003,582.94,0.0017,0.0417,-0.0003,0.1527,0.1181,0.1645,"U.S., equity, SP500, large-cap"
VTI,3.67,-0.0613,0.0353,1.03,4.55,0.0231,0.0134,33.00,3.38000,1.43,0.0003,459.80,0.0067,0.0252,0.0157,0.1388,0.1066,0.1549,"U.S., equity"
QQQ,2.85,-0.0526,0.0571,1.17,10.72,0.0293,0.0180,32.68,33.23000,1.71,0.0020,317.74,0.0134,0.0197,0.0155,0.1314,0.1382,0.2022,"U.S., equity, Nasdaq100"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USRT,1.63,-0.0491,-0.0321,1.00,0.89,0.0168,0.0131,46.78,0.35978,0.59,0.0008,2.94,0.0041,0.0238,-0.0097,0.1454,0.0247,0.0652,"U.S., equity, REITs, real-estate"
EPI,0.12,-0.1242,-0.1647,0.66,0.51,0.0110,0.0071,41.87,1.06000,0.72,0.0085,2.93,0.0000,-0.1110,-0.0493,-0.0719,0.0760,0.1519,"India, equity, quality"
IBDS,1.05,0.0025,-0.0017,0.23,0.05,0.0022,0.0017,58.36,0.72376,0.84,0.0010,2.92,0.0075,0.0923,0.0384,0.0565,0.0157,0.0117,"U.S., fixed-income, corporate-bonds, bonds, in..."


In [10]:
df_stocks.to_pickle(PICKLE_STOCKS_PATH)
print(f'df_stocks saved to {PICKLE_STOCKS_PATH}')

df_etfs.to_pickle(PICKLE_ETFS_PATH)
print(f'df_etfs saved to {PICKLE_ETFS_PATH}')

df_stocks saved to c:\Users\ping\Files_win10\python\py310\stocks\notebooks\..\data\df_finviz_stocks_2025-03-06.pkl
df_etfs saved to c:\Users\ping\Files_win10\python\py310\stocks\notebooks\..\data\df_finviz_etfs_2025-03-06.pkl
